First we get the needed modules

In [ ]:
import http.client
import json
import pandas as pd

Set the needed variables to be able to fecth the data

In [ ]:
# Initialize the variables
cookie = 'wfx_unq=HtDYEhZiAdNZp3y6; bizxCompanyId=semlerit; loginMethodCookieKey=SSO; assertingPartyCookieKey=afwwogyiq.accounts.ondemand.com; route=af85d2f2d68abfd87cd548ed66dd40e711863e69; bizxThemeId=9q44vxh3bw; rxVisitor=1739990854951CTPAMH53EBS2K92CJK9S0MA2OM6KE7R4; rxvt=1739999292880|1739997221512; dtPC=6$597491617_927h-vCRUFHMMORWKPIHCHQIKHGPPSOKMBENCH-0e0; dtSa=false%7C_load_%7C19%7C_onload_%7C-%7C1739997492881%7C597491617_927%7Chttps%3A%2F%2Fperformancemanager.successfactors.eu%2Fsf%2Fstart%2F%7C%7C%7C%2FcompanyCheck%7C; JSESSIONID=2E77F91FD1BEBB3E364F66C06F81CFCC.pc57bcf205; dtCookie=v_4_srv_6_sn_FA94610E03CBB9275A15696A322B76F0_perc_100000_ol_0_mul_1_app-3Ad45ce43a97ce7239_0; BIGipServerhcm57.sapsf.com=344471818.20480.0000; zsessionid=53e71f44-9be9-4f6b-aab8-28f7ee337a04'
csrf_token = 'zkebHzP6JpEpx5Z8gcQ0HGIxbjznMIbVoyENjrAf35U%3d'

Then make the needed definitions

In [ ]:
def get_work_profiles(profile_id):
    global cookie, csrf_token
    
    conn = http.client.HTTPSConnection("performancemanager.successfactors.eu")
    payload = ''
    headers = {
        'accept': 'application/schema-instance+json',
        'accept-language': 'da,en;q=0.9,en-GB;q=0.8,en-US;q=0.7',
        'content-type': 'application/json',
        'cookie': cookie,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36 Edg/127.0.0.0',
        'x-csrf-token': csrf_token
    }
    url = f"/rest/workforce/v1/workProfiles/{profile_id}?navigable=true&$select=id,displayName,displayTitle,assignmentTag,isPrimary,workerType&$expand=directReports($select=id,displayName,displayTitle,assignmentTag,isPrimary,workerType;$expand=workProfileSizeInfo($select=numberOfDirectReports,numberOfMatrixReports,numberOfMatrixSupervisorWorkProfiles,numberOfAssignments,numberOfSupervisorWorkProfiles,totalTeamSize);$top=25;$skip=0),inverseJobRelationships($expand=relatedWorkProfile($select=id,displayName,displayTitle,assignmentTag,isPrimary,workerType;$expand=workProfileSizeInfo($select=numberOfDirectReports,numberOfMatrixReports,numberOfMatrixSupervisorWorkProfiles,numberOfAssignments,numberOfSupervisorWorkProfiles,totalTeamSize));$top=25;$skip=0),workProfileSizeInfo($select=numberOfDirectReports,numberOfMatrixReports,numberOfMatrixSupervisorWorkProfiles,numberOfAssignments,numberOfSupervisorWorkProfiles,totalTeamSize)&=null"
    conn.request("GET", url, payload, headers)
    res = conn.getresponse()
    
    # Check for new cookie and CSRF token in the response headers
    if 'set-cookie' in res.headers:
        new_cookie = res.headers['set-cookie']
        if new_cookie != cookie:
            cookie = new_cookie
            print(f"Updated cookie: {cookie}")
    
    if 'x-csrf-token' in res.headers:
        new_csrf_token = res.headers['x-csrf-token']
        if new_csrf_token != csrf_token:
            csrf_token = new_csrf_token
            print(f"Updated CSRF token: {csrf_token}")
    
    if res.status != 200:
        print(f"Warning: Received HTTP {res.status} - {res.reason}")
        return None
    
    data = res.read()
    return json.loads(data.decode("utf-8"))

def extract_profiles(data, parent_id=None, level=1, counter=[0]):
    if data is None:
        return []
    
    profiles = []
    counter[0] += 1
    profiles.append((data['id'], data['displayName'], data['displayTitle'], parent_id, level, data['workProfileSizeInfo']['numberOfDirectReports'], data['workProfileSizeInfo']['totalTeamSize']))
    print(f"Processing profile: {data['displayName']} (ID: {data['id']}, Level: {level}, Count: {counter[0]})")
    
    if 'directReports' in data and data['workProfileSizeInfo']['numberOfDirectReports'] > 0:
        for report in data['directReports']:
            profiles.extend(extract_profiles(get_work_profiles(report['id']), data['id'], level + 1, counter))
    
    if 'inverseJobRelationships' in data:
        for relationship in data['inverseJobRelationships']:
            related_profile = relationship['relatedWorkProfile']
            profiles.extend(extract_profiles(get_work_profiles(related_profile['id']), data['id'], level + 1, counter))
    
    return profiles

Then we set the inital node, and loop through the data

In [ ]:
# Initial call with the given profile ID
initial_profile_id = "31ADCB529AC2407B95B3EFC28A0DED27"
initial_data = get_work_profiles(initial_profile_id)
profiles_list = extract_profiles(initial_data)

# Print the total number of profiles fetched
print(f"Total number of profiles fetched: {len(profiles_list)}")

The print the count and the result.

Printing the result is a bit redundant, since we save it later in Excel

In [ ]:
# Print the total number of profiles fetched
print(f"Total number of profiles fetched: {len(profiles_list)}")

# Print the list of id, displayName, displayTitle, parentId, and level
for profile_id, displayName, displayTitle, parent_id, level, numberOfDirectReports, totalTeamSize   in profiles_list:
    print(f"ID: {profile_id}, Name: {displayName}, Title: {displayTitle}, Parent ID: {parent_id}, Level: {level}, DirectReports: {numberOfDirectReports}, TeamSize: {totalTeamSize}")

Then save the fetched data in `Excel`

In [ ]:
# Create a DataFrame from the profiles list
df = pd.DataFrame(profiles_list, columns=['ID', 'Name', 'Title', 'Parent ID', 'Level', 'Number of Direct Reports', 'Total Team Size'])


# Save the DataFrame to an Excel file
df.to_excel("./data/profiles.xlsx", index=False)

Then we read the `Excel`file again, and split the names into Firstname, Middlenames and Lastname

In [ ]:
# Define the split_name function
def split_name(full_name):
    name_parts = full_name.split()
    first_name = name_parts[0]
    last_name = name_parts[-1]
    middle_names = " ".join(name_parts[1:-1])
    return first_name, middle_names, last_name

# Read the Excel file
df = pd.read_excel("./data/profiles.xlsx")

# Unfortuneatly, there can be fetched duplicates in the data
df = df.drop_duplicates(keep='first')

# Apply the split_name function to the 'Name' column
df[['First Name', 'Middle Names', 'Last Name']] = df['Name'].apply(lambda x: pd.Series(split_name(x)))

# Save the updated DataFrame to a new Excel file
df.to_excel("./data/profiles_split_names.xlsx", index=False)

# Print a success message
print("The names have been successfully split and saved to output/profiles_split_names.xlsx")

Just for the fun, let's try and create a wordcloud on the firstnames

In [ ]:
# import pandas as pd # Allready imported
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import os

# Define the path to the Excel file
excel_path = "./data/profiles_split_names.xlsx"

# Check if the file exists
if not os.path.exists(excel_path):
    print(f"File not found: {excel_path}")
else:
    # Read the Excel file
    df = pd.read_excel(excel_path)

    # Generate a word cloud from the 'First Name' column
    text_first_name = " ".join(df['First Name'].dropna().astype(str).tolist())
    wordcloud_first_name = WordCloud(width=800, height=400, background_color='white').generate(text_first_name)

    # Display the word cloud for first names
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud_first_name, interpolation='bilinear')
    plt.axis('off')
    plt.title("Word Cloud - First Names")
    plt.show()

    # Generate a word cloud from the 'Title' column
    text_title = " ".join(df['Title'].dropna().astype(str).tolist())
    wordcloud_title = WordCloud(width=800, height=400, background_color='white').generate(text_title)

    # Display the word cloud for titles
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud_title, interpolation='bilinear')
    plt.axis('off')
    plt.title("Word Cloud - Titles")
    plt.show()


Lad os lege lidt med formateringen og farverne i wordcloud'en

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# Define your colors
colors = ["#002D3F", "#0080B5", "#8DC8E8","#A2AAAD","#D0D3D5","#D16B69"] # Semler Colors


# Create the colormap
custom_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)


# Path to your font file
font_path = './font/GreaterTheory.otf'


from collections import Counter

# Check if the file exists
if not os.path.exists(excel_path):
    print(f"File not found: {excel_path}")
else:
    # Read the Excel file
    df = pd.read_excel(excel_path)

    # Generate a word cloud from the 'First Name' column
    names = df["First Name"].dropna().astype(str).str.strip().tolist()
    name_count = Counter(names)
    #text_first_name = " ".join(df['First Name'].dropna().astype(str).tolist())
    wordcloud_first_name = WordCloud(width=800, height=400, background_color='white',colormap=custom_cmap,font_path=font_path).generate_from_frequencies(name_count)

    # Display the word cloud for first names
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud_first_name, interpolation='bilinear')
    plt.axis('off')
    plt.title("Hvilket navn er mest populært i Semler")
    plt.show()

    # Generate a word cloud from the 'Title' column using unique words
    titles = df['Title'].dropna().astype(str).str.strip().tolist()
    title_counts = Counter(titles)
    wordcloud_title = WordCloud(width=800, height=400, background_color='white',colormap=custom_cmap,font_path=font_path).generate_from_frequencies(title_counts)

    # Display the word cloud for titles
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud_title, interpolation='bilinear')
    plt.axis('off')
    plt.title("Hvilken tittel er der flest af i Semler")
    plt.show()

    print(f"Lad os se på de ti mest populære navne i et søjle diagram")

    # Get the 10 most common names
    top_10_names = name_count.most_common(10)

    # Separate the names and their counts
    names, counts = zip(*top_10_names)

    # Create the bar chart
    plt.bar(names, counts, color=custom_cmap(range(len(names))))
    plt.xlabel('Names')
    plt.ylabel('Counts')
    plt.title('Top 10 Names')
    plt.show()


Det sidste søjlediagram er lidt presset og dårligt formateret, lad os prøve at vise det på en anden måde:

In [ ]:
# Sort the data in descending order
sorted_data = sorted(zip(counts, names), reverse=True)
counts, names = zip(*sorted_data)

# Initialize the figure
fig, ax = plt.subplots(figsize=(6, 8))

# Remove axis
ax.axis('off')

# Create the plot
ax.barh(names, counts, color='skyblue')

for i, (value, name) in enumerate(zip(counts, names)):
    ax.text(x=1, y=i, s=f' {name}', ha='left', va='center', fontsize=10)
    ax.text(x=value, y=i, s=f' {int(value)}', ha="left", va="center", fontsize=10)

# Display the plot
plt.show()

Det ser lidt bedre ud, men lad os ændre rækkefølgen og sætte nogle farver på:

In [ ]:

# Sort the data in descending order
sorted_data = sorted(zip(counts, names), reverse=False)
counts, names = zip(*sorted_data)

# Normalize the counts for colormap
norm = plt.Normalize(min(counts), max(counts))
colors = plt.cm.tab10(norm(counts))

# Initialize the figure
fig, ax = plt.subplots(figsize=(6, 8))

# Remove axis
ax.axis('off')

# Create the plot
ax.barh(names, counts, color=colors)

for i, (value, name) in enumerate(zip(counts, names)):
    ax.text(x=1, y=i, s=f' {name}', ha='left', va='center', fontsize=10)
    ax.text(x=value, y=i, s=f' {int(value)}', ha="left", va="center", fontsize=10)

# Display the plot
plt.show()
